In [1]:
from google.cloud import bigquery
import json
import pandas as pd
import os
from ossdirectory import fetch_data
from ossdirectory.fetch import OSSDirectory

In [ ]:
BIGQUERY_PROJECT_NAME = 'opensource-observor'

In [2]:
ossd: OSSDirectory = fetch_data()
address_by_project = {}
for project in ossd.projects:
    project_name = project.get('name', '')
    for addr_entry in project.get('blockchain', []):
        address = addr_entry.get('address', '')
        networks = addr_entry.get('networks', [])
        tags = addr_entry.get('tags', [])
        address_by_project[address] = {
            'networks': networks,
            'tags': tags,
            'name': project_name
        }

In [3]:
with open("s6/data/updated_grants.json", "r") as f:
    grant_data = json.load(f)

address_list = []   
missing_addresses = [] 
    
for g in grant_data:
    
    addresses = []        
    
    wallet = g.pop('wallet_address').strip().lower()
    l2_address = g.pop('l2_addresses').strip().lower()
    if wallet != l2_address:
        wallet = l2_address
        
    if wallet not in address_by_project:
        print("Missing wallet for:", g['project_name'], wallet)
        addresses.append({
            wallet: {
                'networks': ['optimism'],
                'tags': ['wallet']
            }
        })
    else:
        addresses.append({
            wallet: address_by_project.get(wallet)
        })    
    
    if g.get('contract_addresses'):
        contract_addresses = g.pop('contract_addresses')
        if isinstance(contract_addresses, list):
            for a in set(contract_addresses):
                a = a.strip().lower()
                if a not in address_by_project:
                    print("Missing contract for:", g['project_name'], a)
                    missing_addresses.append({'project_name': g['project_name'], 'address': a})
                    addresses.append({
                        a: {
                            'networks': [],
                            'tags': ['contract']
                        }
                    })
                else:
                    addresses.append({a: address_by_project.get(a)})

    g.update({'addresses': addresses})
    address_list.extend([a for addr in addresses for a in addr])

address_list = list(set(address_list))

Missing contract for: Expanding restaking on Optimism 0x73a7fe27fe9545d53924e529acf11f3073841b9e
Missing contract for: Expanding restaking on Optimism 0xe48b4e392e4fc29ac2600c3c8efe0404a15d60d9
Missing wallet for: FrameHack #2 0xe7910f8a4b7efcf2964f017e34e4e2d9aa06edc3
Missing contract for: FrameHack #2 0xe7910f8a4b7efcf2964f017e34e4e2d9aa06edc3
Missing contract for: FrameHack #2 0x00000000fc04c910a0b5fea33b03e0447ad0b0aa
Missing wallet for: Optimism GovQuests 0xa41214012d4462ecbb0724673897ee0dcc0fdf49
Missing contract for: Optimism GovQuests 0xa41214012d4462ecbb0724673897ee0dcc0fdf49
Missing contract for: Optimism GovQuests 0x5d40015034da6cd75411c54dd826135f725c2498
Missing contract for: Optimism GovQuests 0x554866e3654e8485928334e7f91b5afc37d18e04
Missing contract for: Renzo Optimism Season 6 Mission Request - Optimism as base for LRTs 0xbf5495efe5db9ce00f80364c8b423567e58d2110
Missing contract for: Boosting LRT adoption with Compound Finance 0xc9ad14cefb29506534a973f7e0e97e68ece4fa3

In [6]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../../../oso_gcp_credentials.json'
client = bigquery.Client(BIGQUERY_PROJECT_NAME)

addr_list_str = "'" + "','".join(address_list) + "'"
query = f"""
    select
      to_address,
      chain,
      count(*) as count_txns
    from `optimism_superchain_raw_onchain_data.transactions`
    where
        dt > '2024-09-01'
        and to_address in ({addr_list_str})
    group by 1,2

"""

result = client.query(query)
df_bq = result.to_dataframe()
df_bq.to_csv('s6/data/txn_counts.csv')
df_bq = pd.read_csv('s6/data/txn_counts.csv', index_col=0)
df_bq['chain'] = df_bq['chain'].str.replace('op', 'optimism')
df_bq['chain'].value_counts()

/Users/evanmeyer/Desktop/work/kariba-labs/insights/analysis/optimism/govfund_grants/env/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


chain
optimism      65
base          27
mode           6
fraxtal        5
worldchain     4
zora           3
redstone       2
bob            2
mint           1
lisk           1
swan           1
lyra           1
unichain       1
kroma          1
Name: count, dtype: int64

In [7]:
for g in grant_data:
    for a_dict in g['addresses']:
        a = list(a_dict.keys())[0]
        dff_bq = df_bq[df_bq['to_address'] == a]
        if len(dff_bq):
            chains = dff_bq['chain'].unique()
            txns = dff_bq['count_txns'].sum()
            a_dict[a]['networks'].extend(chains)
            a_dict[a].update({'count_txns': txns})
        else:
            a_dict[a].update({'count_txns': 0}) 
        a_dict[a]['networks'] = list(set(a_dict[a]['networks']))

In [8]:
import numpy as np

def convert_numpy(obj):
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, (np.bool_)):
        return bool(obj)
    else:
        return str(obj)

with open("s6/data/updated_grants_reviewed.json", "w") as f:
    json.dump(grant_data, f, indent=2, default=convert_numpy)

In [ ]:
missing_addresses_dict = {}
for missing_address in missing_addresses:
    if missing_address['project_name'] in missing_addresses_dict.keys():
        missing_addresses_dict[missing_address['project_name']].append(missing_address['address'])
    else:
        missing_addresses_dict[missing_address['project_name']] = [missing_address['address']]

with open("s6/data/missing_addresses_log.json", "w") as f:
    json.dump(missing_addresses_dict, f, indent=2, default=convert_numpy)

no_activity_addresses_log = []
for project in grant_data:
    curr_project = {}
    curr_project['project_name'] = project['project_name']
    addresses = project['addresses']
    
    no_activity_addresses = []
    seen_addresses = set()

    for address in addresses:
        addr_key = list(address.keys())[0]
        address_dict = list(address.values())[0]

        if address_dict['count_txns'] == 0 and addr_key not in seen_addresses:
            no_activity_addresses.append(address)
            seen_addresses.add(addr_key)

    if len(no_activity_addresses) == 0:
        continue
    
    curr_project['no_activity_addresses'] = no_activity_addresses
    no_activity_addresses_log.append(curr_project)

with open("s6/data/no_activity_addresses_log.json", "w") as f:
    json.dump(no_activity_addresses_log, f, indent=2, default=convert_numpy)